In [ ]:
import dotenv
dotenv.load_dotenv()

In [2]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    data: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed
event

CalendarEvent(name='Science Fair', data='Friday', participants=['Alice', 'Bob'])

## Example - Chain of thought

In [3]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed
math_reasoning

MathReasoning(steps=[Step(explanation='We start with the equation: \\(8x + 7 = -23\\). Our goal is to solve for \\(x\\). The first step is to isolate the term involving \\(x\\) by getting rid of the constant on the left side of the equation.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='By subtracting 7 from both sides, we can simplify the equation.', output='8x = -30'), Step(explanation='Now, we have \\(8x = -30\\). The next step is to solve for \\(x\\) by dividing both sides of the equation by 8 to isolate \\(x\\).', output='x = \\frac{-30}{8}'), Step(explanation='Simplify the fraction by factoring the numerator and the denominator. The fraction \\(\\frac{-30}{8}\\) can be simplified by dividing both the numerator and the denominator by their greatest common divisor, which is 2.', output='x = \\frac{-15}{4}')], final_answer='x = \\frac{-15}{4}')

In [4]:
print("Solution Steps:")
for step in math_reasoning.steps:
    print(f"Step: {step.explanation}")
    print(f"   → {step.output}\n")

print(f"Final Answer: {math_reasoning.final_answer}")

Solution Steps:
Step: We start with the equation: \(8x + 7 = -23\). Our goal is to solve for \(x\). The first step is to isolate the term involving \(x\) by getting rid of the constant on the left side of the equation.
   → 8x + 7 - 7 = -23 - 7

Step: By subtracting 7 from both sides, we can simplify the equation.
   → 8x = -30

Step: Now, we have \(8x = -30\). The next step is to solve for \(x\) by dividing both sides of the equation by 8 to isolate \(x\).
   → x = \frac{-30}{8}

Step: Simplify the fraction by factoring the numerator and the denominator. The fraction \(\frac{-30}{8}\) can be simplified by dividing both the numerator and the denominator by their greatest common divisor, which is 2.
   → x = \frac{-15}{4}

Final Answer: x = \frac{-15}{4}


## Example - Structured data extraction

In [5]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

text = """Llama 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗ Louis Martin† Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom∗
GenAI, Meta
Abstract
In this work, we develop and release Llama 2, a collection of pretrained and fine-tuned
large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters.
Our fine-tuned LLMs, called Llama 2-Chat, are optimized for dialogue use cases. Our
models outperform open-source chat models on most benchmarks we tested, and based on
our human evaluations for helpfulness and safety, may be a suitable substitute for closedsource models. We provide a detailed description of our approach to fine-tuning and safety
improvements of Llama 2-Chat in order to enable the community to build on our work and
contribute to the responsible development of LLMs."""

class ResearchPaperExtraction(BaseModel):
    title: str
    authors: list[str]
    abstract: str
    keywords: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
        {"role": "user", "content": text}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message.parsed
research_paper

ResearchPaperExtraction(title='Llama 2: Open Foundation and Fine-Tuned Chat Models', authors=['Hugo Touvron', 'Louis Martin', 'Kevin Stone', 'Peter Albert', 'Amjad Almahairi', 'Yasmine Babaei', 'Nikolay Bashlykov', 'Soumya Batra', 'Prajjwal Bhargava', 'Shruti Bhosale', 'Dan Bikel', 'Lukas Blecher', 'Cristian Canton Ferrer', 'Moya Chen', 'Guillem Cucurull', 'David Esiobu', 'Jude Fernandes', 'Jeremy Fu', 'Wenyin Fu', 'Brian Fuller', 'Cynthia Gao', 'Vedanuj Goswami', 'Naman Goyal', 'Anthony Hartshorn', 'Saghar Hosseini', 'Rui Hou', 'Hakan Inan', 'Marcin Kardas', 'Viktor Kerkez', 'Madian Khabsa', 'Isabel Kloumann', 'Artem Korenev', 'Punit Singh Koura', 'Marie-Anne Lachaux', 'Thibaut Lavril', 'Jenya Lee', 'Diana Liskovich', 'Yinghai Lu', 'Yuning Mao', 'Xavier Martinet', 'Todor Mihaylov', 'Pushkar Mishra', 'Igor Molybog', 'Yixin Nie', 'Andrew Poulton', 'Jeremy Reizenstein', 'Rashi Rungta', 'Kalyan Saladi', 'Alan Schelten', 'Ruan Silva', 'Eric Michael Smith', 'Ranjan Subramanian', 'Xiaoqing E

In [6]:
print("="*50)
print(f"Title: {research_paper.title}\n")
print(f"Authors: {', '.join(research_paper.authors)}\n")
print("="*50)
print("Abstract:")
print(research_paper.abstract)
print("="*50)
print("Keywords:")
print(", ".join(research_paper.keywords))
print("="*50)

Title: Llama 2: Open Foundation and Fine-Tuned Chat Models

Authors: Hugo Touvron, Louis Martin, Kevin Stone, Peter Albert, Amjad Almahairi, Yasmine Babaei, Nikolay Bashlykov, Soumya Batra, Prajjwal Bhargava, Shruti Bhosale, Dan Bikel, Lukas Blecher, Cristian Canton Ferrer, Moya Chen, Guillem Cucurull, David Esiobu, Jude Fernandes, Jeremy Fu, Wenyin Fu, Brian Fuller, Cynthia Gao, Vedanuj Goswami, Naman Goyal, Anthony Hartshorn, Saghar Hosseini, Rui Hou, Hakan Inan, Marcin Kardas, Viktor Kerkez, Madian Khabsa, Isabel Kloumann, Artem Korenev, Punit Singh Koura, Marie-Anne Lachaux, Thibaut Lavril, Jenya Lee, Diana Liskovich, Yinghai Lu, Yuning Mao, Xavier Martinet, Todor Mihaylov, Pushkar Mishra, Igor Molybog, Yixin Nie, Andrew Poulton, Jeremy Reizenstein, Rashi Rungta, Kalyan Saladi, Alan Schelten, Ruan Silva, Eric Michael Smith, Ranjan Subramanian, Xiaoqing Ellen Tan, Binh Tang, Ross Taylor, Adina Williams, Jian Xiang Kuan, Puxin Xu, Zheng Yan, Iliyan Zarov, Yuchen Zhang, Angela Fan, Me

## Example - Moderation

In [8]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "How do I prepare for a job interview?"}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed
compliance

ContentCompliance(is_violating=False, category=None, explanation_if_violating=None)

In [10]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Category(str, Enum):
    violence = "violence"
    sexual = "sexual"
    self_harm = "self_harm"

class ContentCompliance(BaseModel):
    is_violating: bool
    category: Optional[Category]
    explanation_if_violating: Optional[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Determine if the user input violates specific guidelines and explain if they do."},
        {"role": "user", "content": "Can you tell me how to make the drug?"}
    ],
    response_format=ContentCompliance,
)

compliance = completion.choices[0].message.parsed
compliance

ContentCompliance(is_violating=True, category=<Category.self_harm: 'self_harm'>, explanation_if_violating='Providing instructions on how to make drugs could lead to self-harm, illegal activities, and endangering the health and safety of individuals.')

## Streaming

In [11]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI

class EntitiesModel(BaseModel):
  attributes: List[str]
  colors: List[str]
  animals: List[str]

client = OpenAI()

with client.beta.chat.completions.stream(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "Extract entities from the input text"},
      {
          "role": "user",
          "content": "The quick brown fox jumps over the lazy dog with piercing blue eyes",
      },
  ],
  response_format=EntitiesModel,
) as stream:
  for event in stream:
      if event.type == "content.delta":
          if event.parsed is not None:
              # Print the parsed data as JSON
              print("content.delta parsed:", event.parsed)
      elif event.type == "content.done":
          print("content.done")
      elif event.type == "error":
          print("Error in stream:", event.error)

final_completion = stream.get_final_completion()
print("Final completion:", final_completion)

content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {'attributes': []}
content.delta parsed: {'attributes': []}
content.delta parsed: {'attributes': ['quick']}
content.delta parsed: {'attributes': ['quick']}
content.delta parsed: {'attributes': ['quick', 'brown']}
content.delta parsed: {'attributes': ['quick', 'brown']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing', 'blue']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing', 'blue']}
content.delta parsed: {'attributes': ['quick', 'brown', 'lazy', 'piercing', 'blue']}
content.delta parsed: {'attributes': ['quick', 'brown'